In [ ]:


import torch
import torch.nn as nn
import torchvision.datasets
from torchvision import transforms , datasets
#import torchvision.transforms as transforms
from torch.optim import lr_scheduler
!pip install torchsummary
from torchsummary import summary

import matplotlib.pyplot as plt
from torch.nn.utils import spectral_norm

from torch.utils.data import Subset

import os
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

import torch.nn as nn
import torch.functional as F
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from torchvision.models import inception_v3
inception_model = inception_v3(pretrained=True)
#inception_model.load_state_dict(torch.load("inception_v3_google-1a9a5a14.pth"))
inception_model.to(device)
inception_model = inception_model.eval() # Evaluation mode
inception_model.fc = nn.Identity()
def fid(num : int , dataset , inception_model , generator):
    fake_features = []
    real_features = []
    for i in range(num):
        z = torch.normal(mean = 0 , std = 1 , size = (1 ,1, 16 , 16)).to(device)
        idx = torch.randint(0 , dataset.__len__() , (1,1)).item()
        real_img = dataset[idx].to(device)
        fake_img = generator(z)

        real_img = torch.nn.functional.pad(real_img , (127, 128  , 127 ,128))
        fake_img  = torch.nn.functional.pad(fake_img ,(127 , 128 ,127 ,128))


        real_img = real_img.unsqueeze(dim = 0)
        #fake_img = fake_img.unsqueeze(dim = 0)

        fake_features.append(inception_model(fake_img))
        real_features.append(inception_model(real_img))

    real_matrix = torch.stack(real_features)
    fake_matrix = torch.stack(fake_features)

    real_mean = torch.mean(real_matrix , dim = 0)
    fake_mean = torch.mean(fake_matrix , dim = 0)
    real_matrix = real_matrix.squeeze(dim  = 1)
    fake_matrix = fake_matrix.squeeze(dim  = 1)

    real_cov = torch.cov( torch.transpose(real_matrix,0 , 1 ))
    fake_cov = torch.cov( torch.transpose(fake_matrix, 0 , 1))

    return (torch.square(torch.norm((real_mean - fake_mean), p ='fro')) + torch.trace(real_cov + fake_cov - 2*torch.sqrt(real_cov*fake_cov)))




In [ ]:
class Discriminator(nn.Module):
  def __init__(self , num , dim):
    super(Discriminator, self ).__init__()

    # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True,
    # padding_mode='zeros', device=None, dtype=None)

    #Hout =⌊  ( Hin +2×padding[0]−dilation[0]×(kernel_size[0]−1)−1 / stride[0]) + 1  ⌋

    #input is 64x64 and output is a sigmoid function

    self.conv1 = spectral_norm((nn.Conv2d(3 , 64 ,kernel_size = 3 ,stride =  2 ,padding =  1 )))
    self.batchnorm1 = nn.BatchNorm2d(64)
    self.elu1 = nn.LeakyReLU(0.1, inplace = False)
    self.dropout1 = nn.Dropout2d(0.375)

    self.conv2 = spectral_norm(nn.Conv2d(64 , 128,kernel_size=  3 ,stride =  2,padding =  1 ))
    self.batchnorm2 = nn.BatchNorm2d(128)
    self.elu2 =  nn.LeakyReLU(0.1, inplace = False)
    self.dropout2 = nn.Dropout2d(0.375)

    self.conv3 = spectral_norm((nn.Conv2d(128, 256 ,kernel_size =  3 ,stride =  2 ,padding =  1 )))
    self.batchnorm3 = nn.BatchNorm2d(256)
    self.elu3 = nn.LeakyReLU(0.1, inplace = False)
    self.dropout3 = nn.Dropout2d(0.375)

    self.conv4 = spectral_norm(nn.Conv2d(256 , 512 ,kernel_size =  3 ,stride =  2 , padding = 1 ))
    self.batchnorm4 = nn.BatchNorm2d(512)
    self.elu4 =  nn.LeakyReLU(0.1, inplace = False)
    self.dropout4 = nn.Dropout2d(0.375)


    self.leakyrelu= (nn.LeakyReLU(negative_slope =0.2))
    self.simple = spectral_norm(nn.Linear(8192, 1))

    self.dropout = nn.Dropout(p=0.60)
    self.weights_init()

    # same layers as that of the generator
  def weights_init(m):
    print("init")
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
        #self.weight_init(m.weight.data, mean=0.0, std=0.08)
        nn.init.xavier_uniform_(m.weight)
        #nn.init.normal_(m.weight, mean=0.0, std=0.02, generator=None)
  def forward(self , img):
    h1 =self.elu1((self.conv1(img)))
    #h1 = self.dropout1(h1)
    h2 =self.elu2((self.conv2(h1)))
    #h2 = self.dropout2(h2)
    h3 =self.elu3((self.conv3(h2)))
    #h3 = self.dropout3(h3)
    h4 =self.elu4((self.conv4(h3)))
    #h4 = self.dropout4(h4)


    h6 = torch.flatten(h4 , start_dim = 1)
    #h6 = self.dropout(h6)
    h7 = self.simple(h6)
    return self.leakyrelu(h7) , h6












In [ ]:
class Generator(nn.Module):
  # it takes in an input of a tensor (2-D) of size 8x8 in the paper it is a 1-D array , but for simplicity of convolution's dimensions i am taking y = x
  #Hout =(Hin − 1 )×stride[0]−2×padding[0]+dilation[0]×(kernel_size[0]−1)+output_padding[0]+1

  #Wout =(Win − 1 )×stride[1]−2×padding[1]+dilation[1]×(kernel_size[1]−1)+output_padding[1]+1



  def __init__(self , num , dim ):
    super(Generator, self).__init__()

  # i will be inputting a tensor of size (batch_size , 1(channels) , num(y dimension) , num(x dimension))
  # and i want a output of (batch_size , 3(channels) , dim(y dimension) , dim(x dimension))


#torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
  #output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros', device=None, dtype=None)

    #take a noise of 100 size , then make it into 1024*4*4 , then reshape it into (1 , 1024 , 4 , 4)

    # i am assuming that the covolutions with stride 1 are the reason behind the patternized outputs ... bcz i dont have anything else to blame
    #that on and also the notebook which i saw was working had this only
    self.linear = nn.Linear(256 , 16384)
    self.elul =  nn.LeakyReLU(0.1, inplace = False)

    self.convt1 = nn.ConvTranspose2d(in_channels =1024 , out_channels = 512 , kernel_size = 3 , stride = 2 , padding = 1, output_padding = 1)
    self.batchnorm1 = nn.BatchNorm2d(512)
    self.elu1 =  nn.LeakyReLU(0.1, inplace = False)

    self.convt2 = nn.ConvTranspose2d(512 , 256,kernel_size =  3 ,stride =  2  , padding = 1 , output_padding = 1 )
    self.batchnorm2 = nn.BatchNorm2d(256)
    self.elu2 =  nn.LeakyReLU(0.1, inplace = False)


    self.convt3 = nn.ConvTranspose2d(256, 128 ,kernel_size =  3,stride =  2 , padding = 1 , output_padding = 1   )
    self.batchnorm3 = nn.BatchNorm2d(128)
    self.elu3 = nn.LeakyReLU(0.1, inplace = False)


    self.convt5 = nn.ConvTranspose2d(128 , 3 ,kernel_size = 3 ,stride =  2 ,padding =  1  , output_padding = 1)
    self.batchnorm5 = nn.BatchNorm2d(1)
    self.relu5 = nn.ReLU(inplace = False)
    self.tanh = nn.Tanh()

    self.weights_init()
  def weights_init(m):
    print("initdone")
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
        #self.weight_init(m.weight.data, mean=0.0, std=0.08)
        nn.init.xavier_uniform_(m.weight)
        #nn.init.normal_(m.weight, mean=0.0, std=0.02, generator=None)
  def forward(self , z):
    z = torch.flatten(z , start_dim = 1)
    z1 = self.elul(self.linear(z))

    z2 = z1.reshape(-1 , 1024 , 4, 4)
    h1 =self.elu1(self.batchnorm1(self.convt1(z2)))
    h2 =self.elu2((self.convt2(h1)))
    h3 =self.elu3((self.convt3(h2)))

    h5 =self.relu5((self.convt5(h3)))

    h6 = self.tanh(h5)

    return h6







In [ ]:

# first create a dataset , and a dataloader
trans = transforms.Compose([

    transforms.Resize((64,64)),
    torchvision.transforms.ConvertImageDtype(torch.float32)

    ])
trans_for_mnist =  transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64,64)),
    torchvision.transforms.ConvertImageDtype(torch.float32)

    ])


#dataset class
subset_size = 440
class img_dataset(Dataset):
  def __init__(self , img_dir , trans , subset_size):
    self.img_dir = img_dir
    self.transform = trans
    self.files = os.listdir(self.img_dir)
    self.data = []
    self.subset_size = subset_size
    for idx , file in enumerate(self.files):
        if (idx > subset_size):
            break
        img = read_image(os.path.join(self.img_dir , file))
        img = trans(img)
        self.data.append(img)

  def __len__(self):

    return self.subset_size
  def __getitem__(self ,idx ):
    img = self.data[idx]
    return img







In [ ]:

loss_func = nn.BCELoss()
generator = Generator(4 , 64)
discriminator = Discriminator (4, 64)

#trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=trans_for_mnist)
dataset = img_dataset("/kaggle/input/gan-large-area-png", trans , subset_size)
#dataset = trainset
batch = 64







#device thing
# Assuming your model is named 'model'

generator = generator.to(device)
#discriminator.spectral_norm()
discriminator = discriminator.to(device)

#dataset = dataset.to(device)


g_optim = torch.optim.Adam(generator.parameters() , lr = 0.00002 , betas = (0 , 0.9) )
d_optim = torch.optim.Adam(discriminator.parameters() , lr = 0.0000375, betas = (0 , 0.9) )

#d_optim = torch.optim.(discriminator.parameters() , lr = 0.00005, betas = (0.5 , 0.999))
#g_optim = torch.optim.Adam(generator.parameters() , lr = 0.00005, betas = (0.5 , 0.999))

g_scheduler = lr_scheduler.ExponentialLR(g_optim, gamma=0.998)
d_scheduler = lr_scheduler.ExponentialLR(d_optim, gamma=0.998)

subset  = Subset(dataset , list(range(subset_size)))
data_loader = DataLoader(subset, batch_size = batch , shuffle = True , pin_memory = True)


#shapes
# z = (batch, 1 , 8 , 8)
z = torch.normal(mean = 0 , std = 1 , size = (batch ,1, 16 , 16)).to(device)
print("shape of generator output = " + str((generator(z).shape)))
print("shape of discriminator output = " + str(discriminator(generator(z))[0].shape))


print(dataset.__len__())

for i in range(5):
    img = dataset[i]
    plt.imshow(img.permute(1,2,0))
    plt.show()


In [ ]:
print(dataset.__len__())


In [ ]:
summary(generator , input_size = (1,16,16))
summary(discriminator , input_size = (3,64,64))
summary(inception_model, input_size = (3, 299 , 299))
#remember the discriminator parameters shown here are double of the actual number bcz of the spectral norm thing
generator = nn.DataParallel(generator)
discriminator = nn.DataParallel(discriminator)

In [ ]:
epochs = 3000
iter = 4

ones = torch.ones(batch,1).to(device)
zeroes = torch.zeros(batch,1).to(device)
fidlist = []
generator_loss = []
discriminator_loss = []
epoch_count = []
discriminator_loss_part1 = []
discriminator_loss_part2 = []
feature_loss_list = []
clip_value = 1


In [ ]:
import torchvision.utils as vutils

print(ones.dtype)
generator.train()
discriminator.train()
feature_loss =  torch.nn.MSELoss()
factor = 0
for epoch in tqdm(range(epochs)):



  factor = factor*0.995
  #print(epoch)
  epoch_count.append(epoch)
  generator_loss.append(0)
  discriminator_loss.append(0)
  discriminator_loss_part1.append(0)
  discriminator_loss_part2.append(0)
  feature_loss_list.append(0)
  fidlist.append(fid(50 , dataset , inception_model , generator).item())

  for batch_idx , (data) in enumerate(data_loader):
    if(batch_idx % iter == 0):
        generator.train()
        discriminator.eval()

        #print("gen----------------------------------------------" , batch_idx)
        batch_size = (data.shape[0])
        data = data.to(device)
        z = torch.normal(mean = 0 , std = 1 ,size = (batch_size , 1 , 16,16) ).to(device)
        discriminator_output_generated , discriminator_features_generated = discriminator(generator(z))
        discriminator_output_real, discriminator_features_real = discriminator(data)

        f_loss = feature_loss(discriminator_features_generated , discriminator_features_real)
        feature_loss_list[epoch] += f_loss.item()
        g_loss = torch.mean(-1*(discriminator_output_generated))

        g_loss = (1-factor)*g_loss+ factor*f_loss
        generator_loss[epoch] += torch.mean(g_loss).item()
        d_optim.zero_grad()

        g_optim.zero_grad()

        g_loss.backward()
        #torch.nn.utils.clip_grad_norm_(generator.parameters(), clip_value)
        g_optim.step()
    else:
        generator.eval()
        discriminator.train()
        #print("dis" , batch_idx)
        #print(batch_idx)
        batch_size = (data.shape[0])
        data = data.to(device)


        z = torch.normal(mean = 0 , std = 1 ,size = (batch_size , 1 , 16,16)).to(device)
        discriminator_output_real = discriminator(data)[0]
        discriminator_output_generated = discriminator(generator(z))[0]
        part1 = (discriminator_output_real)
        part2 = ( discriminator_output_generated)
        #d_loss = loss_func(discriminator_output_real, ones) + loss_func( discriminator_output_generated, zeroes)
        d_loss = torch.mean(-1*(part1 - part2))
        discriminator_loss[epoch] += (d_loss).item()/(iter -1)
        discriminator_loss_part1[epoch] += -1*torch.mean(part1).item()/(iter-1)

        discriminator_loss_part2[epoch] += torch.mean(part2).item()/(iter-1)
        d_optim.zero_grad()
        d_loss.backward()

        d_optim.step()

   #torch.nn.utils.clip_grad_norm_(generator.parameters(), clip_value)

    if(epoch%2 == 0):


            if batch_idx == 0:
                with torch.no_grad():
                    generator.eval()
                    sample_z = torch.normal(mean=0, std=1, size=(1, 1, 16, 16)).to(device)
                    generated_img = generator(sample_z).cpu()

                    # Convert the image tensor to [0, 1] range and save it
                    img_path = f'images/epoch_{epoch}_batch_{batch_idx}.png'
                    vutils.save_image(generated_img, img_path, normalize=True)


  #g_scheduler.step()
  #d_scheduler.step()



    #print(str(data.min()) + "   " + str(data.max()))
    #print(str(generator(z).min()) + "  " + str(generator(z).max()))

  print(f"epoch - : {epoch_count[epoch]},Generator loss - : {generator_loss[epoch]}, Discriminator - : {discriminator_loss[epoch]} , g_lr = {g_optim.param_groups[0]['lr']} , d_lr = {d_optim.param_groups[0]['lr']}")


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt  # Import matplotlib for saving images
from tqdm import tqdm

# Assuming 'discriminator' is your pre-trained model

# Define the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained model
model = discriminator
model.eval()
model = model.to(device)

# Define loss function (for example, maximizing activation of a certain layer)
loss_activation = []
loss_function = nn.MSELoss()

# Initialize input image (random noise or blank image)
input_img = torch.randn(1, 3, 64, 64, device=device, requires_grad=True)  # or initialize with zeros or random noise
input_img = input_img.to(device)

# Define optimizer
optimizer = optim.Adam([input_img], lr=0.0001)

# Optimization loop
num_iterations = 100  # Increase iterations for more refined result
for i in tqdm(range(num_iterations)):
    optimizer.zero_grad()
    output = model(input_img)[0]

    # Maximize the activation of the chosen layer (example)
    loss = -torch.mean(model(input_img)[0])
    loss_activation.append(loss.item())

    loss.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:  # Adjust as needed to save images periodically
        # Convert optimized input image to PIL image for visualization
        output_img = transforms.ToPILImage()(input_img.squeeze().detach().cpu())

        # Save the image using matplotlib
        plt.imshow(output_img)
        plt.axis('off')
        plt.savefig(f'output_{i+1}.png', bbox_inches='tight')
        plt.close()

# Plot loss vs iterations2
plt.figure()
plt.plot(range(num_iterations), loss_activation, label='Loss Activation')
plt.xlabel('Iterations')
plt.ylabel('Loss Activation')
plt.title('Loss Activation vs Iterations')
plt.grid(True)
plt.savefig('loss_vs_iterations.png', bbox_inches='tight')
plt.close()


In [ ]:

import matplotlib.pyplot as plt

# Sample data for demonstration
x_values = epoch_count
y1_values = generator_loss
y2_values = discriminator_loss
y3_values = discriminator_loss_part1
y4_values = discriminator_loss_part2
y5_values = feature_loss_list

# Plotting the first line
plt.plot(x_values, y1_values, label='generator_loss')

# Plotting the second line
plt.plot(x_values, y2_values, label='discriminator_loss')

plt.plot(x_values, y3_values, label='discriminator_loss_part1')


# Plotting the second line
plt.plot(x_values, y4_values, label='discriminator_loss_part2')

plt.plot(x_values , y5_values , label= 'feature_loss')

# Adding labels and title
plt.xlabel('epoch')
plt.ylabel('losses')
plt.title('Loss Graphs')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()

clipped_discriminator = [40 if x > 40 else -40 if x < -40 else x for x in discriminator_loss]
plt.plot(epoch_count , clipped_discriminator , label = 'clipped_discriminator')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.title('Discriminator_loss')

plt.legend()


plt.show()


plt.plot(x_values, y1_values, label='generator_loss')

# Plotting the second line
plt.plot(x_values, y2_values, label='discriminator_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss graph')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()


# Sample data for demonstration
x_values = epoch_count
y1_values = discriminator_loss_part1
y2_values = discriminator_loss_part2
# Plotting the first line
plt.plot(x_values, y1_values, label='discriminator_loss_part1')


# Plotting the second line
plt.plot(x_values, y2_values, label='discriminator_loss_part2')

# Adding labels and title
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('discriminator_losses_broken_down')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()



In [ ]:
y1_values = fidlist
plt.plot(x_values, y1_values, label='fid score')

# Plotting the second line

plt.xlabel('epoch')
plt.ylabel('score')
plt.title('FID score ')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()

y1_values = [min(x , 100) for x in fidlist]
plt.plot(x_values, y1_values, label='cropped fid score')

# Plotting the second line

plt.xlabel('epoch')
plt.ylabel('score')
plt.title('Cropped FID score ')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()


In [ ]:
y1_values = feature_loss_list
plt.plot(x_values, y1_values, label='feature_losses')

plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Feature Loss graph')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()

#cropped graph

y1_values= [min(x , 50) for x in feature_loss_list]
plt.plot(x_values, y1_values, label='cropped feature_losses')

plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Cropped Feature Loss graph')

# Adding a legend to differentiate lines
plt.legend()

# Display the plot
plt.show()



In [ ]:
import pickle

# Save a list to a file

with open('generator_loss.pkl', 'wb') as f:
    pickle.dump(generator_loss, f)


with open('discriminator_loss.pkl', 'wb') as f:
    pickle.dump(discriminator_loss, f)





In [ ]:
torch.cuda.empty_cache()
base = generator(z)[0]
lista = []
for i in range(1000):
        z = torch.normal(mean = 0 , std = 1, size = (32 , 1 , 16 , 16)).to(device)
        img = generator(z)[0]
        diff = torch.sum(base - img)
        lista.append(diff.item())


In [ ]:
print(len(lista))
plt.scatter(lista, range(0, len(lista) ))

plt.show()

In [ ]:
for i in range(10):

        z = torch.normal(mean = 0 , std = 1 , size = (batch ,1, 16 , 16)).to(device)
        img = generator(z)[0].to('cpu').detach()
        img  = img.permute(1,2,0)
        print(torch.min(img) , "    " , torch.max(img))
        plt.imshow(img)
        plt.show()

In [ ]:
fidlist

In [ ]:
len(fidlist)

In [ ]:
fidlist[2995:2999]

In [ ]:
min(fidlist)

In [ ]:
!zip -r file.zip /kaggle/working


In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')
